In [1]:
import json
import os
import pandas as pd
import sqlite3

# Function to read JSON files
def read_json(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")  # Check if file exists
    with open(file_path, 'r') as file:
        return json.load(file)  # Load JSON data from the file

# Define file paths
customer_file_path = 'C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_customer.json'
branch_file_path = 'C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_branch.json'
credit_card_file_path = 'C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_credit.json'

# Read data from JSON files
customers = read_json(customer_file_path)
branches = read_json(branch_file_path)
credit_cards = read_json(credit_card_file_path)

# Function to format phone numbers
def format_phone_number(x):
    if pd.isnull(x) or not isinstance(x, str) or len(x) != 10:
        return x  # Return original value if it is not a valid phone number
    return f"({x[:3]}){x[3:6]}-{x[6:]}"  # Format the phone number

# Transformation function for customer data
def transform_customers(data):
    df = pd.DataFrame(data)  # Convert JSON data to DataFrame
    df['FIRST_NAME'] = df['FIRST_NAME'].str.title()  # Capitalize first name
    df['MIDDLE_NAME'] = df['MIDDLE_NAME'].str.lower()  # Lowercase middle name
    df['LAST_NAME'] = df['LAST_NAME'].str.title()  # Capitalize last name
    df['FULL_STREET_ADDRESS'] = df['STREET_NAME'] + ", " + df['APT_NO']  # Combine street name and apt number
    df['CUST_PHONE'] = df['CUST_PHONE'].astype(str).apply(format_phone_number)  # Format phone numbers
    return df.drop_duplicates(subset=['SSN', 'CREDIT_CARD_NO'])  # Drop duplicate rows based on SSN and credit card number

# Transformation function for branch data
def transform_branches(data):
    df = pd.DataFrame(data)  # Convert JSON data to DataFrame
    df['BRANCH_PHONE'] = df['BRANCH_PHONE'].astype(str).apply(format_phone_number)  # Format phone numbers
    df['BRANCH_ZIP'] = df['BRANCH_ZIP'].fillna('999999')  # Fill missing ZIP codes with a default value
    return df.drop_duplicates(subset=['BRANCH_CODE'])  # Drop duplicate rows based on branch code

# Transformation function for credit card data
def transform_credit_cards(data):
    df = pd.DataFrame(data)  # Convert JSON data to DataFrame
    df['TIMEID'] = pd.to_datetime(df[['DAY', 'MONTH', 'YEAR']]).dt.strftime('%Y%m%d')  # Create a TIMEID column
    return df.drop_duplicates(subset=['TRANSACTION_ID'])  # Drop duplicate rows based on transaction ID

# Transform data
transformed_customers = transform_customers(customers)
transformed_branches = transform_branches(branches)
transformed_credit_cards = transform_credit_cards(credit_cards)

# Function to load data into SQLite database
def load_to_database(df, table_name):
    conn = sqlite3.connect('creditcard_capstone.db')  # Connect to SQLite database or MySQL for me this way is just quicker and easier 
    df.to_sql(table_name, conn, if_exists='replace', index=False)  # Load DataFrame to SQL table
    conn.close()  # Close the database connection

# Load transformed data into the database
load_to_database(transformed_customers, 'customers')
load_to_database(transformed_branches, 'branches')
load_to_database(transformed_credit_cards, 'credit_cards')
